<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/3_analysis_gender_org.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\TABLES/data_fund_region_country_and_org_clean_dummies_col_replace_grp.csv')#.drop(['Addresses'], axis=1)
data = data.loc[data['funding_yes'] == 1]
data.head(2)

,Unnamed: 0,wos_ID,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,east asia and pacific,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
1,1,wos:000921279800001,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
2,2,wos:000909001200001,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0


In [ ]:
data.count().head(10)

Unnamed: 0                 577
wos_ID                     577
funding_yes                577
open_access_yes            577
Funding Name Preferred     575
europe and central asia    125
north america              431
africa                       2
sub-saharan africa          20
east asia and pacific       78
dtype: int64

# The gender disparities of Marburg Virus Diseases publications

## all female proportion per studies

### import data

In [ ]:
data_female_prop = pd.read_csv(f'../TABLES_old/author_gender_list_count_dummies.csv')
data_female_prop = data_female_prop.rename(columns={'UT (Unique WOS ID)':'wos_ID'})
data_female_prop = data_female_prop.loc[data_female_prop['funding_yes'] == 1]
data_female_prop['wos_ID'] = data_female_prop['wos_ID'].str.lower()
data_female_prop.head(2)

,wos_ID,Publication Year,Funding Orgs,funding_yes,female,male,unclassified,unknown,number_of_author,%_female,%_male,%_unclass_name,female_to_male_ratio
1,wos:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,2.0,9.0,0.0,0.0,11.0,18.181818,81.818182,0.0,0.222222
2,wos:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,0.0,1.0,0.0,0.0,1.0,0.000000,100.000000,0.0,0.000000


In [ ]:
data_concat = data_female_prop[['wos_ID','%_female']].merge(data, on = 'wos_ID', how='left')
data_concat

,wos_ID,%_female,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
0,wos:000921279800001,18.181818,1,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
1,wos:000909001200001,0.000000,2,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0
2,wos:000777045900001,7.142857,3,1,1,National Research Foundation of Korea (NRF) - ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,national research foundation of korea (nrf) - ...,"green accepted, gold",10.0,12.0,28.0,2022.0
3,wos:000922983600001,0.000000,5,1,0,Intravacc,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,intravacc,NaN,NaN,NaN,NaN,NaN
4,wos:000905136600002,57.894737,6,1,0,Intramural Research Program of the Vaccine Res...,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,intramural research program of the vaccine res...,NaN,1.0,1.0,1.0,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,wos:000231482100016,0.000000,747,1,1,NIAID NIH HHS(United States Department of Heal...,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,niaid nih hhs [r01ai46963] funding source: med...,bronze,90.0,1.0,11.0,2005.0
573,wos:000227629300024,0.000000,754,1,1,NCRR NIH HHS(United States Department of Healt...,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,ncrr nih hhs [p20-rr15635] funding source: med...,green published,12.0,NaN,4.0,2005.0
574,wos:000224540900066,6.666667,763,1,1,NIAID NIH HHS(United States Department of Heal...,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,niaid nih hhs [r21 ai059172] funding source: m...,"green published, green accepted",293.0,NaN,20.0,2004.0
575,wos:000182631100037,0.000000,785,1,1,NHLBI NIH HHS(United States Department of Heal...,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,"nhlbi nih hhs [f32 hl067623, p01 hl051670, r01...",green published,104.0,NaN,3.0,2003.0


In [ ]:
# data_concat

### proportion all

In [ ]:
dataT = data_concat.T

dataT['countries'] = dataT.index

female_std_list = []
female_median_list = []
female_mean_list = []
female_sum_list = []
for country in data_concat.columns:
    female_sum_list.append(data_concat.dropna(subset=[country])['%_female'].sum())
    female_mean_list.append(data_concat.dropna(subset=[country])['%_female'].mean())

# rename countries names on columns
countries = list(dataT.index)
female_sum  = female_sum_list
female_mean = female_mean_list

dataT_sum = pd.DataFrame({'countries': countries, '%_female_sum': female_sum})
dataT_mean = pd.DataFrame({'countries': countries, '%_female_mean': female_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['%_female_count'] = dataT_concat['%_female_sum']/dataT_concat['%_female_mean']

#####
dataT_concat_all_female = dataT_concat[['countries','%_female_sum','%_female_mean','%_female_count']].set_index('countries')

# save data
dataT_concat_all_female.to_csv(f'../TABLES/region_countries_org_%_female_sum_mean_count.csv')
dataT_concat_all_female

,%_female_sum,%_female_mean,%_female_count
countries,,,
wos_ID,19552.002467,33.885620,577.0
%_female,19552.002467,33.885620,577.0
Unnamed: 0,19552.002467,33.885620,577.0
funding_yes,19552.002467,33.885620,577.0
open_access_yes,19552.002467,33.885620,577.0
...,...,...,...
Open Access Designations,17379.309294,34.828275,499.0
"Times Cited, All Databases",18718.586847,34.220451,547.0
180 Day Usage Count,8170.183570,34.619422,236.0


In [ ]:
len(female_sum)

522

In [ ]:
len(female_mean)

522

## first authors gender

### import data

In [ ]:
first_author_gender = pd.read_csv(f'../DATA_PREPROCESSING/first_author_gender.csv')
first_author_gender = first_author_gender.reset_index().rename(columns={'UT (Unique WOS ID)':'wos_ID'})
first_author_gender = first_author_gender.loc[first_author_gender['funding_yes'] == 1]
first_author_gender['wos_ID'] = first_author_gender['wos_ID'].str.lower()
first_author_gender.head(2)

,index,wos_ID,Publication Year,Funding Orgs,funding_yes,ga_gender
1,1,wos:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,male
2,2,wos:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,male


In [ ]:
data_concat_first = first_author_gender[['wos_ID','ga_gender']].merge(data, on = 'wos_ID', how='left')
data_concat_first.head(2)

,wos_ID,ga_gender,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
0,wos:000921279800001,male,1,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
1,wos:000909001200001,male,2,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0


In [ ]:
data_concat_first['ga_gender'].value_counts()

male            311
female          246
unclassified     18
unknown           2
Name: ga_gender, dtype: int64

In [ ]:
first_gender_list_all = data_concat_first.reset_index().groupby(['ga_gender']).count().T
# countries_oa_list_all.rename(columns = {0:'all_is_no_oa',
#                               1:'all_is_oa'}, inplace = True)

first_gender_list_all['all_total'] = first_gender_list_all['male']+ first_gender_list_all['female'] + first_gender_list_all['unclassified']+ first_gender_list_all['unknown']

for col in first_gender_list_all.columns:
    first_gender_list_all['%'+col] = first_gender_list_all[col]/first_gender_list_all['all_total']*100

first_gender_list_all['%_diff(%male-%female)'] = first_gender_list_all['%male'] - first_gender_list_all['%female']

# save data
first_gender_list_all.to_csv(f'../TABLES/region_countries_org_gender_first_authors.csv')

first_gender_list_all

ga_gender,female,male,unclassified,unknown,all_total,%female,%male,%unclassified,%unknown,%all_total,%_diff(%male-%female)
index,246,311,18,2,577,42.634315,53.899480,3.119584,0.346620,100.0,11.265165
wos_ID,246,311,18,2,577,42.634315,53.899480,3.119584,0.346620,100.0,11.265165
Unnamed: 0,246,311,18,2,577,42.634315,53.899480,3.119584,0.346620,100.0,11.265165
funding_yes,246,311,18,2,577,42.634315,53.899480,3.119584,0.346620,100.0,11.265165
open_access_yes,246,311,18,2,577,42.634315,53.899480,3.119584,0.346620,100.0,11.265165
...,...,...,...,...,...,...,...,...,...,...,...
Open Access Designations,217,271,9,2,499,43.486974,54.308617,1.803607,0.400802,100.0,10.821643
"Times Cited, All Databases",235,294,16,2,547,42.961609,53.747715,2.925046,0.365631,100.0,10.786106
180 Day Usage Count,104,125,6,1,236,44.067797,52.966102,2.542373,0.423729,100.0,8.898305
Since 2013 Usage Count,240,300,18,2,560,42.857143,53.571429,3.214286,0.357143,100.0,10.714286


## last authors gender

### import data

In [ ]:
last_author_gender = pd.read_csv(f'../DATA_PREPROCESSING/last_author_gender.csv')
last_author_gender = last_author_gender.reset_index().rename(columns={'UT (Unique WOS ID)':'wos_ID'})
last_author_gender = last_author_gender.loc[last_author_gender['funding_yes'] == 1]
last_author_gender['wos_ID'] = last_author_gender['wos_ID'].str.lower()
last_author_gender.head(2)

,index,wos_ID,Publication Year,Funding Orgs,funding_yes,Author Full Names,gender
1,1,wos:000921279800001,2023.0,National Institutes of Health (NIH) [U19AI1427...,1,"heyes, james",male
2,2,wos:000909001200001,2023.0,Russian Science Foundation; [22-13-00036],1,"moskalik, mikhail yu.",male


In [ ]:
data_concat_last = last_author_gender[['wos_ID','gender']].merge(data, on = 'wos_ID', how='left')
data_concat_last.head(2)

,wos_ID,gender,Unnamed: 0,funding_yes,open_access_yes,Funding Name Preferred,europe and central asia,north america,africa,sub-saharan africa,...,fulbright foundation - usa,world bank - multilateral,china association for science and technology - china,zhejiang university - china,Funding Orgs,Open Access Designations,"Times Cited, All Databases",180 Day Usage Count,Since 2013 Usage Count,Publication Year
0,wos:000921279800001,male,1,1,1,National Institutes of Health (NIH)(United Sta...,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,national institutes of health (nih) [u19ai1427...,hybrid,NaN,NaN,NaN,2023.0
1,wos:000909001200001,male,2,1,1,Russian Science Foundation(Russian Science Fou...,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,russian science foundation; [22-13-00036],"gold, green accepted",NaN,3.0,3.0,2023.0


In [ ]:
data_concat_last['gender'].value_counts()

male            454
female          110
unclassified     11
unknown           2
Name: gender, dtype: int64

In [ ]:
last_gender_list_all = data_concat_last.reset_index().groupby(['gender']).count().T
# countries_oa_list_all.rename(columns = {0:'all_is_no_oa',
#                               1:'all_is_oa'}, inplace = True)

last_gender_list_all['all_total'] = last_gender_list_all['male']+ last_gender_list_all['female'] + last_gender_list_all['unclassified']+ last_gender_list_all['unknown']

for col in last_gender_list_all.columns:
    last_gender_list_all['%'+col] = last_gender_list_all[col]/last_gender_list_all['all_total']*100

last_gender_list_all['%_diff(%male-%female)'] = last_gender_list_all['%male'] - last_gender_list_all['%female']

# save data
last_gender_list_all.to_csv(f'../TABLES/region_countries_org_gender_last_authors.csv')

last_gender_list_all

gender,female,male,unclassified,unknown,all_total,%female,%male,%unclassified,%unknown,%all_total,%_diff(%male-%female)
index,110,454,11,2,577,19.064125,78.682842,1.906412,0.346620,100.0,59.618718
wos_ID,110,454,11,2,577,19.064125,78.682842,1.906412,0.346620,100.0,59.618718
Unnamed: 0,110,454,11,2,577,19.064125,78.682842,1.906412,0.346620,100.0,59.618718
funding_yes,110,454,11,2,577,19.064125,78.682842,1.906412,0.346620,100.0,59.618718
open_access_yes,110,454,11,2,577,19.064125,78.682842,1.906412,0.346620,100.0,59.618718
...,...,...,...,...,...,...,...,...,...,...,...
Open Access Designations,98,394,6,1,499,19.639279,78.957916,1.202405,0.200401,100.0,59.318637
"Times Cited, All Databases",106,429,10,2,547,19.378428,78.427788,1.828154,0.365631,100.0,59.049360
180 Day Usage Count,49,181,4,2,236,20.762712,76.694915,1.694915,0.847458,100.0,55.932203
Since 2013 Usage Count,107,440,11,2,560,19.107143,78.571429,1.964286,0.357143,100.0,59.464286


In [ ]:
xxxxx

NameError: name 'xxxxx' is not defined

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.head(2)

In [ ]:
dataT = data_fund.T

dataT['countries'] = dataT.index

female_std_list = []
female_median_list = []
female_mean_list = []
female_sum_list = []
for country in data.columns:
    female_sum_list.append(data_fund.dropna(subset=[country])['Times Cited, All Databases'].sum())
    female_mean_list.append(data_fund.dropna(subset=[country])['Times Cited, All Databases'].mean())

# rename countries names on columns
countries = list(dataT.index)
female_sum  = female_sum_list
female_mean = female_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'fund_female_sum': female_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'fund_female_mean': female_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['fund_female_count'] = dataT_concat['fund_female_sum']/dataT_concat['fund_female_mean']

#####*****************
dataT_concat_fund_tca = dataT_concat[['countries','fund_female_sum','fund_female_mean','fund_female_count']].set_index('countries')

# save data
# dataT_concat_fund.to_csv(f'../TABLES/countries_fund_female_sum_mean_count.csv')
dataT_concat_fund_tca

### proportion no fund research

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund.head(2)

In [ ]:
dataT = data_no_fund.T

dataT['countries'] = dataT.index

female_std_list = []
female_median_list = []
female_mean_list = []
female_sum_list = []
for country in data.columns:
    female_sum_list.append(data_no_fund.dropna(subset=[country])['Times Cited, All Databases'].sum())
    female_mean_list.append(data_no_fund.dropna(subset=[country])['Times Cited, All Databases'].mean())

# rename countries names on columns
countries = list(dataT.index)
female_sum  = female_sum_list
female_mean = female_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'no_fund_female_sum': female_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'no_fund_female_mean': female_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['no_fund_female_count'] = dataT_concat['no_fund_female_sum']/dataT_concat['no_fund_female_mean']

#####

dataT_concat_no_fund_tca = dataT_concat[['countries','no_fund_female_sum','no_fund_female_mean','no_fund_female_count']].set_index('countries')

# save data
# dataT_concat_no_fund.to_csv(f'../TABLES/countries_no_fund_female_sum_mean.csv')
dataT_concat_no_fund_tca

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_pub_female_list = pd.concat([dataT_concat_all_tca,
                                    dataT_concat_fund_tca,
                                    dataT_concat_no_fund_tca],
                                    axis=1)
# save data
concat_data_pub_female_list.to_csv(f'../TABLES/concat_data_pub_female_list.csv')
concat_data_pub_female_list

### significance

In [ ]:
df_fund = data[['funding_yes','Times Cited, All Databases']]
df_fund

In [ ]:
df_fund.count()

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund = df_fund.dropna()
df_fund

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund.describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 0].describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 1].describe()

In [ ]:
#------------

In [ ]:
fund_TCA = df_fund.loc[df_fund['funding_yes'] == 1]['Times Cited, All Databases'].to_list()
fund_TCA[:15]

In [ ]:
no_fund_TCA = df_fund.loc[df_fund['funding_yes'] == 0]['Times Cited, All Databases'].to_list()

In [ ]:
test_sig(fund_TCA, no_fund_TCA)

In [ ]:
import plotly.express as px
fig = px.box(df_fund, x="funding_yes", y="Times Cited, All Databases", color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()

## '180 Day Usage Count'

### proportion all

In [ ]:
dataT = data.T

dataT['countries'] = dataT.index

last_180_Day_Usage_Count_std_list = []
last_180_Day_Usage_Count_median_list = []
last_180_Day_Usage_Count_mean_list = []
last_180_Day_Usage_Count_sum_list = []
for country in data.columns:
    last_180_Day_Usage_Count_sum_list.append(data.dropna(subset=[country])['180 Day Usage Count'].sum())
    last_180_Day_Usage_Count_mean_list.append(data.dropna(subset=[country])['180 Day Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
last_180_Day_Usage_Count_sum  = last_180_Day_Usage_Count_sum_list
last_180_Day_Usage_Count_mean = last_180_Day_Usage_Count_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'all_180_Day_Usage_Count_sum': last_180_Day_Usage_Count_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'all_180_Day_Usage_Count_mean': last_180_Day_Usage_Count_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['all_180_Day_Usage_Count_count'] = dataT_concat['all_180_Day_Usage_Count_sum']/dataT_concat['all_180_Day_Usage_Count_mean']

#####


dataT_concat_all_180_day = dataT_concat[['countries','all_180_Day_Usage_Count_sum','all_180_Day_Usage_Count_mean']].set_index('countries')

# save data
# dataT_concat_all.to_csv(f'../TABLES/countries_all_180_Day_Usage_Count_sum_mean.csv')
dataT_concat_all_180_day

In [ ]:
len(last_180_Day_Usage_Count_sum)

In [ ]:
len(last_180_Day_Usage_Count_mean)

### proportion fund research

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.shape

In [ ]:
dataT = data_fund.T

dataT['countries'] = dataT.index

last_180_Day_Usage_Count_std_list = []
last_180_Day_Usage_Count_median_list = []
last_180_Day_Usage_Count_mean_list = []
last_180_Day_Usage_Count_sum_list = []
for country in data.columns:
    last_180_Day_Usage_Count_sum_list.append(data_fund.dropna(subset=[country])['180 Day Usage Count'].sum())
    last_180_Day_Usage_Count_mean_list.append(data_fund.dropna(subset=[country])['180 Day Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
last_180_Day_Usage_Count_sum  = last_180_Day_Usage_Count_sum_list
last_180_Day_Usage_Count_mean = last_180_Day_Usage_Count_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'fund_180_Day_Usage_Count_sum': last_180_Day_Usage_Count_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'fund_180_Day_Usage_Count_mean': last_180_Day_Usage_Count_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['fund_180_Day_Usage_Count_count'] = dataT_concat['fund_180_Day_Usage_Count_sum']/dataT_concat['fund_180_Day_Usage_Count_mean']

#####


dataT_concat_fund_180_day = dataT_concat[['countries','fund_180_Day_Usage_Count_sum','fund_180_Day_Usage_Count_mean','fund_180_Day_Usage_Count_count']].set_index('countries')

# save data
# dataT_concat_fund.to_csv(f'../TABLES/countries_fund_last_180_Day_Usage_Count_sum_mean.csv')
dataT_concat_fund_180_day

### proportion no fund research

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund

In [ ]:
dataT = data_no_fund.T

dataT['countries'] = dataT.index

last_180_Day_Usage_Count_std_list = []
last_180_Day_Usage_Count_median_list = []
last_180_Day_Usage_Count_mean_list = []
last_180_Day_Usage_Count_sum_list = []
for country in data.columns:
    last_180_Day_Usage_Count_sum_list.append(data_no_fund.dropna(subset=[country])['180 Day Usage Count'].sum())
    last_180_Day_Usage_Count_mean_list.append(data_no_fund.dropna(subset=[country])['180 Day Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
last_180_Day_Usage_Count_sum  = last_180_Day_Usage_Count_sum_list
# female_std = female_std_list
# female_median = female_median_list
last_180_Day_Usage_Count_mean = last_180_Day_Usage_Count_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'no_fund_180_Day_Usage_Count_sum': last_180_Day_Usage_Count_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'no_fund_180_Day_Usage_Count_mean': last_180_Day_Usage_Count_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['no_fund_180_Day_Usage_Count_count'] = dataT_concat['no_fund_180_Day_Usage_Count_sum']/dataT_concat['no_fund_180_Day_Usage_Count_mean']

#####


dataT_concat_no_fund_180_day = dataT_concat[['countries','no_fund_180_Day_Usage_Count_sum','no_fund_180_Day_Usage_Count_mean','no_fund_180_Day_Usage_Count_count']].set_index('countries')

# save data
# dataT_concat_no_fund.to_csv(f'../TABLES/countries_no_fund_last_180_Day_Usage_Count_sum_mean_count.csv')
dataT_concat_no_fund_180_day

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_pub_180_day_list = pd.concat([dataT_concat_all_180_day,
                                    dataT_concat_fund_180_day,
                                    dataT_concat_no_fund_180_day],
                                    axis=1)
# save data
concat_data_pub_180_day_list.to_csv(f'../TABLES/concat_data_pub_180_day_usage_list.csv')
concat_data_pub_180_day_list

### significance

In [ ]:
df_fund = data[['funding_yes','180 Day Usage Count']]
df_fund

In [ ]:
df_fund.count()

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund = df_fund.dropna()
df_fund

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund.describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 0].describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 1].describe()

In [ ]:
#------------

In [ ]:
fund_TCA = df_fund.loc[df_fund['funding_yes'] == 1]['180 Day Usage Count'].to_list()
fund_TCA[:15]

In [ ]:
no_fund_TCA = df_fund.loc[df_fund['funding_yes'] == 0]['180 Day Usage Count'].to_list()

In [ ]:
test_sig(fund_TCA, no_fund_TCA)

In [ ]:
import plotly.express as px
fig = px.box(df_fund, x="funding_yes", y="180 Day Usage Count", color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()

## 'Since 2013 Usage Count'

### proportion all

In [ ]:
dataT = data.T

dataT['countries'] = dataT.index

Since_2013_Usage_Coun_std_list = []
Since_2013_Usage_Coun_median_list = []
Since_2013_Usage_Coun_mean_list = []
Since_2013_Usage_Coun_sum_list = []
for country in data.columns:
    Since_2013_Usage_Coun_sum_list.append(data.dropna(subset=[country])['Since 2013 Usage Count'].sum())
    Since_2013_Usage_Coun_mean_list.append(data.dropna(subset=[country])['Since 2013 Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
Since_2013_Usage_Coun_sum  = Since_2013_Usage_Coun_sum_list
Since_2013_Usage_Coun_mean = Since_2013_Usage_Coun_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'all_since_2013_Usage_Coun_sum': Since_2013_Usage_Coun_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'all_since_2013_Usage_Coun_mean': Since_2013_Usage_Coun_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['all_since_2013_Usage_Coun_count'] = dataT_concat['all_since_2013_Usage_Coun_sum']/dataT_concat['all_since_2013_Usage_Coun_mean']

#####


dataT_concat_all_since2013usage = dataT_concat[['countries','all_since_2013_Usage_Coun_sum','all_since_2013_Usage_Coun_mean','all_since_2013_Usage_Coun_count']].set_index('countries')

# save data
# dataT_concat_all.to_csv(f'../TABLES/countries_Since_2013_Usage_Count_sum_mean.csv')
dataT_concat_all_since2013usage

In [ ]:
len(Since_2013_Usage_Coun_sum)

In [ ]:
len(Since_2013_Usage_Coun_mean)

### proportion fund research

In [ ]:
data_fund = data.loc[data['funding_yes'] == 1]
data_fund.shape

In [ ]:
dataT = data_fund.T

dataT['countries'] = dataT.index

Since_2013_Usage_Coun_std_list = []
Since_2013_Usage_Coun_median_list = []
Since_2013_Usage_Coun_mean_list = []
Since_2013_Usage_Coun_sum_list = []
for country in data.columns:
    Since_2013_Usage_Coun_sum_list.append(data_fund.dropna(subset=[country])['Since 2013 Usage Count'].sum())
    Since_2013_Usage_Coun_mean_list.append(data_fund.dropna(subset=[country])['Since 2013 Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
Since_2013_Usage_Coun_sum  = Since_2013_Usage_Coun_sum_list
Since_2013_Usage_Coun_mean = Since_2013_Usage_Coun_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'fund_since_2013_Usage_Coun_sum': Since_2013_Usage_Coun_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'fund_since_2013_Usage_Coun_mean': Since_2013_Usage_Coun_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['fund_since_2013_Usage_Coun_count'] = dataT_concat['fund_since_2013_Usage_Coun_sum']/dataT_concat['fund_since_2013_Usage_Coun_mean']

#####


dataT_concat_fund_since2013usage = dataT_concat[['countries','fund_since_2013_Usage_Coun_sum','fund_since_2013_Usage_Coun_mean','fund_since_2013_Usage_Coun_count']].set_index('countries')

# save data
# dataT_concat_fund.to_csv(f'../TABLES/countries_fund_Since_2013_Usage_Count_sum_mean.csv')
dataT_concat_fund_since2013usage

### proportion no fund research

In [ ]:
data_no_fund = data.loc[data['funding_yes'] == 0]
data_no_fund.shape

In [ ]:
dataT = data_no_fund.T

dataT['countries'] = dataT.index

Since_2013_Usage_Coun_std_list = []
Since_2013_Usage_Coun_median_list = []
Since_2013_Usage_Coun_mean_list = []
Since_2013_Usage_Coun_sum_list = []
for country in data.columns:
    Since_2013_Usage_Coun_sum_list.append(data_no_fund.dropna(subset=[country])['Since 2013 Usage Count'].sum())
    Since_2013_Usage_Coun_mean_list.append(data_no_fund.dropna(subset=[country])['Since 2013 Usage Count'].mean())

# rename countries names on columns
countries = list(dataT.index)
Since_2013_Usage_Coun_sum  = Since_2013_Usage_Coun_sum_list
Since_2013_Usage_Coun_mean = Since_2013_Usage_Coun_mean_list

dataT_sum = pd.DataFrame({'countries': countries, 'no_fund_since_2013_Usage_Coun_sum': Since_2013_Usage_Coun_sum})
dataT_mean = pd.DataFrame({'countries': countries, 'no_fund_since_2013_Usage_Coun_mean': Since_2013_Usage_Coun_mean_list})

# ********
dataT_concat = dataT.reset_index().merge(dataT_sum, on = 'countries', how='left')
dataT_concat = dataT_concat.merge(dataT_mean, on = 'countries', how='left')

dataT_concat['no_fund_since_2013_Usage_Coun_count'] = dataT_concat['no_fund_since_2013_Usage_Coun_sum']/dataT_concat['no_fund_since_2013_Usage_Coun_mean']

#####


dataT_concat_no_fund_since2013usage = dataT_concat[['countries','no_fund_since_2013_Usage_Coun_sum','no_fund_since_2013_Usage_Coun_mean','no_fund_since_2013_Usage_Coun_count']].set_index('countries')

# save data
# dataT_concat_no_fund.to_csv(f'../TABLES/countries_no_fund_Since_2013_Usage_Count_sum_mean.csv')
dataT_concat_no_fund_since2013usage

### concat and save data

In [ ]:
# concatenate dataframes on index
concat_data_pub_female_list = pd.concat([dataT_concat_all_since2013usage,
                                    dataT_concat_fund_since2013usage,
                                    dataT_concat_no_fund_since2013usage],
                                    axis=1)
# save data
concat_data_pub_female_list.to_csv(f'../TABLES/concat_data_pub_since2013usage_list.csv')
concat_data_pub_female_list

### significance

In [ ]:
df_fund = data[['funding_yes','Since 2013 Usage Count']]
df_fund

In [ ]:
df_fund.count()

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund = df_fund.dropna()
df_fund

In [ ]:
df_fund.reset_index().groupby(['funding_yes']).count()

In [ ]:
df_fund.describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 0].describe()

In [ ]:
df_fund.loc[df_fund['funding_yes'] == 1].describe()

In [ ]:
#------------

In [ ]:
fund_TCA = df_fund.loc[df_fund['funding_yes'] == 1]['Since 2013 Usage Count'].to_list()
fund_TCA[:15]

In [ ]:
no_fund_TCA = df_fund.loc[df_fund['funding_yes'] == 0]['Since 2013 Usage Count'].to_list()

In [ ]:
test_sig(fund_TCA, no_fund_TCA)

In [ ]:
import plotly.express as px
fig = px.box(df_fund, x="funding_yes", y="Since 2013 Usage Count", color="funding_yes")
# setting the y-axis range from 0 to 50
fig.update_layout(yaxis_range=[-2,100])
fig.update_layout(font_size=20)

fig.show()